Multiple model - hyperparamter combinations were tried. Broadly, transfer learning was applied on base models of Xception and InceptionResNetV2. The code with best results only is shown. 

In [ ]:
import pandas as pd
import numpy as np
import os
import zipfile
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow.keras.backend as K
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping,ModelCheckpoint
from keras import Input, metrics, losses
import warnings
warnings.filterwarnings('ignore')
LOCAL_PATH = '../data'
KAGGLE_PATH = '/kaggle'
PATH = KAGGLE_PATH
cwd = os.getcwd()

In [ ]:
base_dir = os.path.join(PATH, "input/dogs-vs-cats-redux-kernels-edition")
train_dir = os.path.join(base_dir, "train.zip")
test_dir = os.path.join(base_dir, "test.zip")

with zipfile.ZipFile(train_dir,"r") as z:
    z.extractall()

with zipfile.ZipFile(test_dir,"r") as z:
    z.extractall()

Training data is put under train/cat and train/dog with respect to their labels. Test data is put under test/test_files to make it compatible with flow_from_directory.

In [ ]:
def make_dir(dir):
    for d in dir:
        new_path = cwd+'/train/'+d
        os.makedirs(new_path,exist_ok=True)
    print('make_dir complete')
        
def move_files():
    n_train_cats = 0
    n_train_dogs = 0
    train_path = cwd+'/train/'
    test_path = cwd+'/test/'
    
    pic_paths = os.listdir(train_path)
    for i in tqdm(pic_paths):
        if 'jpg' in i:
            if 'dog' in i:
                os.rename(train_path+i,train_path+'dog/'+i )
                n_train_dogs+=1
            elif 'cat' in i:
                os.rename(train_path+i,train_path+'cat/'+i )
                n_train_cats+=1
                
    pic_paths = os.listdir(test_path)
    os.makedirs(test_path+'test_files',exist_ok=True)
    for i in tqdm(pic_paths):
        if 'jpg' in i:
            os.rename(test_path+i, test_path+'test_files/'+i)
    return n_train_cats, n_train_dogs

In [ ]:
make_dir(['cat', 'dog'])

In [ ]:
n_train_cats, n_train_dogs = move_files()
# mv  ./dog.* ./dog

In [ ]:
n_train_cats, n_train_dogs

Multiple combinations of data augmentation were tried along with different input image shapes and varying batch sizes. 

In [ ]:
train_path = cwd+'/train/'
test_path = cwd+'/test/'
size = (300, 300)

train_data_gen = ImageDataGenerator(
#     rotation_range=20,
#     shear_range=0.2,
#     zoom_range=0.2,
#     horizontal_flip=True,
    validation_split=0.2)

val_data_gen = ImageDataGenerator(
    validation_split=0.2)

train_data = train_data_gen.flow_from_directory(
        train_path,
        target_size=size,
        batch_size=64,
        seed=29,
        subset='training',
        class_mode='binary'
)

val_data = val_data_gen.flow_from_directory(
        train_path,
        target_size=size,
        batch_size=64,
        seed=29,
        class_mode='binary',
        subset='validation'
)

In [ ]:
train_data.class_indices

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
        image = train_data[1][0][i]
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow((image/255))
        plt.title(int(train_data[1][1][i]))
        plt.axis("off")

In [ ]:
base_model = keras.applications.Xception(
    include_top=False,
    input_shape=(300, 300, 3),
    weights="imagenet"
)
# base_model = keras.applications.InceptionResNetV2(
#     include_top=False,
#     input_shape=(300, 300, 3),
#     weights="imagenet"
# )

In [ ]:
# for layer in base_model.layers[:-5]:
#     layer.trainable = False
# base_model.summary()
base_model.trainable = False

In [ ]:
inputs = Input(shape=(300, 300, 3))
scale_layer = keras.layers.Rescaling(scale=1 / 127.5, offset=-1)
x = scale_layer(inputs)
x = base_model(x, training=False)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
outputs = Dense(1, activation='sigmoid')(x)
model = Model(inputs, outputs)

# outputs = Dense(2,activation='softmax')(x)

model.summary()

In [ ]:
# model_checkpoint_callback  = ModelCheckpoint(filepath = '../working/weights.{epoch:02d}-{val_loss:.2f}.hdf5', save_best_only = True, mode = 'min')
model_checkpoint_callback  = ModelCheckpoint(filepath = '../working/best_model.hdf5', save_best_only = True, mode = 'min')
model_stopping_callback  = EarlyStopping(patience=10,mode='min',restore_best_weights=True)
callbacks = [model_stopping_callback, model_checkpoint_callback]

Different combinations of momentum and learning rates were tried.

In [ ]:
sgd_optimizer = tf.keras.optimizers.SGD(momentum = 0.9, nesterov = True)
model.compile(optimizer=sgd_optimizer,loss=losses.BinaryCrossentropy(),metrics=[metrics.BinaryAccuracy()])

In [ ]:
history = model.fit(train_data,epochs=20,
                    validation_data=val_data,
                    callbacks=callbacks)

In [ ]:
# model_dir = os.path.join(PATH, "input/best-model/weights.12-0.03.hdf5")
# best_model = load_model('./weights.12-0.02.hdf5')

In [ ]:
epochs = list(range(1,len(history.history['loss'])+1))
plt.plot(epochs, history.history['loss'], epochs, history.history['val_loss'])
plt.legend(('Training loss','Validation loss'))
plt.show()

In [ ]:
best_model = load_model('./best_model.hdf5')
best_model.trainable = True
best_model.summary()

In [ ]:
sgd_optimizer = tf.keras.optimizers.SGD(learning_rate=1e-5, momentum = 0.9, nesterov = True)
best_model.compile(optimizer=sgd_optimizer,loss=losses.BinaryCrossentropy(),metrics=[metrics.BinaryAccuracy()])

In [ ]:
history = best_model.fit(train_data,epochs=20,
                    validation_data=val_data,
                    callbacks=callbacks)

In [ ]:
epochs = list(range(1,len(history.history['loss'])+1))
plt.plot(epochs, history.history['loss'], epochs, history.history['val_loss'])
plt.legend(('Training loss','Validation loss'))
plt.show()

In [ ]:
test_data_gen = ImageDataGenerator()

test_data = val_data_gen.flow_from_directory(
        test_path,
        target_size=size,
        shuffle=False,
        class_mode=None)

In [ ]:
best_model = load_model('./best_model.hdf5')
# y_pred = tf.nn.sigmoid(best_model.predict(test_data)).numpy().flatten()
y_pred = best_model.predict(test_data).flatten()

In [ ]:
# submission = pd.DataFrame({'id':pd.Series(test_data.filenames),'label':pd.Series(y_pred.clip(min=0.02,max=0.98))})
submission = pd.DataFrame({'id':pd.Series(test_data.filenames),'label':pd.Series(y_pred)})
submission['id'] = submission.id.str.extract('(\d+)')
submission['id']=pd.to_numeric(submission['id'])

In [ ]:
submission.to_csv('submission.csv',index=False)